- Usaremos diversas bibliotecas: nltk, pandas e scikit-learn
- Lembre se que algumas bibliotecas devem ser instaladas!
- Usei "pip install *nome da biblioteca*" pra fazer a instalação

In [12]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

In [15]:
dataset = pd.read_csv(r'D:\mathe\Documents\worspace_pessoal\UFU\segundo_semestre\information_retrieval\tp4\Tweets_Mg.csv',encoding='utf-8')
# dataset = pd.read_csv(r"C:\Users\Miani\Dropbox\Rodrigo\Trabalho\UFU\Disciplinas\Organização e Recuperação da Informação\Exercícios\TP\TP4\Tweets_Mg.csv",encoding='utf-8')

In [ ]:
## Funções auxiliares para remoção de stopwords

In [13]:
nltk.download("stopwords")
stopwords = list(set(nltk.corpus.stopwords.words("portuguese")))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mathe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
def removeStopWords(sentence: str) -> str:
    words: list[str] = sentence.split(' ')
    cleanSentence: str = ""
    for word in words:
        if(word not in stopwords):
            cleanSentence += ' ' + word
    return ''.join(cleanSentence)    

In [36]:
def removeAllStopWords(sentences: list[str]) -> list[str]:
    cleanSentences: list[str] = []
    for sentence in sentences:
        cleanSentences.append(removeStopWords(sentence))
    return cleanSentences
        

In [17]:
dataset.head()

,Unnamed: 0,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,NaN,NaN,Brasil,Leonardo C Schneider,LeoCSchneider,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,NaN,NaN,NaN,Ana estudando,estudandoconcur,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,NaN,NaN,NaN,Emily,Milly777,0,Negativo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
dataset.count()

Unnamed: 0                   8199
Created At                   8199
Text                         8199
Geo Coordinates.latitude      104
Geo Coordinates.longitude     104
User Location                5489
Username                     8199
User Screen Name             8199
Retweet Count                8199
Classificacao                8199
Observação                      1
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [19]:
# Vamos contar quantos tweets de cada tipo existem: neutro, positivo e negativo.

In [20]:
dataset[dataset.Classificacao=='Neutro'].count()

Unnamed: 0                   2453
Created At                   2453
Text                         2453
Geo Coordinates.latitude      102
Geo Coordinates.longitude     102
User Location                1712
Username                     2453
User Screen Name             2453
Retweet Count                2453
Classificacao                2453
Observação                      0
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [21]:
dataset[dataset.Classificacao=='Positivo'].count()

Unnamed: 0                   3300
Created At                   3300
Text                         3300
Geo Coordinates.latitude        1
Geo Coordinates.longitude       1
User Location                2118
Username                     3300
User Screen Name             3300
Retweet Count                3300
Classificacao                3300
Observação                      1
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [22]:
dataset[dataset.Classificacao=='Negativo'].count()

Unnamed: 0                   2446
Created At                   2446
Text                         2446
Geo Coordinates.latitude        1
Geo Coordinates.longitude       1
User Location                1659
Username                     2446
User Screen Name             2446
Retweet Count                2446
Classificacao                2446
Observação                      0
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [23]:
# Precisamos criar váriaveis diferentes para armazenar os tweets e a sua classificação

In [24]:
tweets_com_stopwords = dataset['Text'].values
classes = dataset['Classificacao'].values

['���⛪ @ Catedral de Santo Antônio - Governador Valadares/MG https://t.co/JSbKamIqUJ'
 '� @ Governador Valadares, Minas Gerais https://t.co/B3ThIDJCSf'
 '�� @ Governador Valadares, Minas Gerais https://t.co/dPkgzVR2Qw' ...
 'Trio é preso suspeito de roubo, tráfico e abuso sexual em Uberlândia https://t.co/zaQbXRRJWc'
 'Trio é preso suspeito de roubo, tráfico e abuso sexual em Uberlândia: Um dos autores teria molestado vítima de… https://t.co/lQ8cTSNftA'
 'Trio suspeito de roubo de cargas é preso em Santa Luzia (MG) https://t.co/0INgJcMtZb #R7MG #RecordTVMinas']


In [12]:
## Removendo stopwords

In [ ]:
## Acurácia dos tweets com stopwords

In [68]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
freq_tweets = vectorizer.fit_transform(tweets_com_stopwords)
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=10)

In [69]:
metrics.accuracy_score(classes,resultados)

0.8896206854494451

In [70]:
## Acurácia dos tweets sem stopwords

In [71]:
tweets_sem_stopwords = removeAllStopWords(tweets_com_stopwords)

In [72]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
freq_tweets = vectorizer.fit_transform(tweets_sem_stopwords)
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=10)

In [73]:
metrics.accuracy_score(classes,resultados)

0.9007195999512135

In [74]:
print(metrics.classification_report(classes,resultados))

              precision    recall  f1-score   support

    Negativo       0.89      0.95      0.92      2446
      Neutro       0.87      0.84      0.85      2453
    Positivo       0.94      0.91      0.92      3300

    accuracy                           0.90      8199
   macro avg       0.90      0.90      0.90      8199
weighted avg       0.90      0.90      0.90      8199



In [65]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   Negativo  Neutro  Positivo   All
Real                                      
Negativo      2319     112        15  2446
Neutro         199    2061       193  2453
Positivo        87     208      3005  3300
All           2605    2381      3213  8199
